Start a new Jupyter Notebook to complete these tasks. This week, you'll be combining two different datasets.

First, you'll work with a dataset containing the number of primary care physicians per county for each county in the United States. It was obtained from the Area Health Resources File, published by the [Health Resources and Services Administration](https://data.hrsa.gov/topics/health-workforce/ahrf). This data is contained in the file `primary_care_physicians.csv`.

Second, the file `population_by_county.csv` contains the Census Bureau's 2019 population estimates for each US County. It also contains a column `urban`. The `urban` column uses data from the National Bureau of Economic Research to classify each county as either urban or rural. The U.S. Office of Management and Budget designates counties as metropolitan (a core urban area of 50,000 or more population), micropolitan (an urban core of at least 10,000 but less than 50,000 population), or neither. Here, a county is considered "urban" if it is part of a metropolitan or micropolitan area and "rural" if it is not.

 1. First, import the primary care physicians dataset (`primary_care_physicians.csv`) into a data frame named `physicians`. 
 2. Filter `physicians` down to just the counties in Tennessee. Save the filtered dataframe back to `physicians`. Verify that the resulting dataframe has 95 rows.
 3. Look at the distribution of the number of primary care physicians. What do you notice?
 4. Now, import the population by county dataset (`population_by_county.csv`) into a DataFrame named `population`.
 5. Merge the `physicians` DataFrame with the `population` DataFrame. Keep only the values for Tennessee. When you merge, be sure the include both the `population` and `urban` columns in the merged results. Save the result of the merge back to `physicians`.
 6. How many Tennessee counties are considered urban?
 7. The State Health Access Data Assistance Center (SHADAC) (https://www.shadac.org/) classifies counties into three groups based on the number of residents per primary care physician. First, counties with fewer than 1500 residents per primary care physician are considered to have an "adequate" supply. Counties with at least 1500 residents but fewer than 3500 residents per primary care physician are considered to have a "moderately inadequate" supply, and counties with at least 3500 residents per primary care physician are considered to have a "low inadequate" supply. How many counties in Tennessee are in each group? 
 8. Does there appear to be any detectable relationship between whether a county is urban or rural and its supply of primary care physicians?

In [93]:
import pandas as pd

In [94]:
physicians = pd.read_csv('../data/primary_care_physicians.csv')

In [95]:
physicians.head()

,FIPS,state,county,primary_care_physicians
0,1001,Alabama,Autauga,26.0
1,1003,Alabama,Baldwin,153.0
2,1005,Alabama,Barbour,8.0
3,1007,Alabama,Bibb,12.0
4,1009,Alabama,Blount,12.0


In [96]:
physicians = physicians[physicians['state']=='Tennessee']

In [97]:
physicians.shape

(95, 4)

In [98]:
physicians['primary_care_physicians'].describe()

count     95.000000
mean      51.042105
std      129.311426
min        0.000000
25%        4.500000
50%       12.000000
75%       26.500000
max      806.000000
Name: primary_care_physicians, dtype: float64

Most counties have 5-40 primary care physicians but some counties have zero while some have hundreds.

In [99]:
population = pd.read_csv('../data/population_by_county.csv')

In [100]:
population.head(5)

,FIPS,population,county,state,urban
0,17051,21565,Fayette County,ILLINOIS,Rural
1,17107,29003,Logan County,ILLINOIS,Rural
2,17165,23994,Saline County,ILLINOIS,Rural
3,17097,701473,Lake County,ILLINOIS,Urban
4,17127,14219,Massac County,ILLINOIS,Rural


In [101]:
population = population[population['state']=='TENNESSEE']

In [102]:
population = population.replace(' County','',regex=True)
#population['county'].str.rsplit(' ', n = 1, expand = True)
#physicians = pd.merge(left = physicians,
#               right = population.loc[population['state']=="TENNESSEE"][['FIPS', 'population', 'urban']], 
#         on="FIPS"
#        )

In [103]:
population = population[['population','county','urban']]

In [104]:
physicians = pd.merge(left = physicians, right = population, left_on = 'county', right_on = 'county')

In [105]:
physicians.head()

,FIPS,state,county,primary_care_physicians,population,urban
0,47001,Tennessee,Anderson,39.0,76061,Urban
1,47003,Tennessee,Bedford,15.0,48292,Rural
2,47005,Tennessee,Benton,3.0,16140,Rural
3,47007,Tennessee,Bledsoe,1.0,14836,Rural
4,47009,Tennessee,Blount,90.0,129927,Urban


In [106]:
physicians.urban.value_counts()

Rural    57
Urban    38
Name: urban, dtype: int64

There are 38 urban counties

In [107]:
physicians["pop_per_pcp"] = physicians.population/physicians.primary_care_physicians

In [108]:
adequate = physicians.loc[physicians['pop_per_pcp']<1500]
adequate.shape

(14, 7)

14 counties have adequate pcp coverage

In [109]:
moderately_inadequate = physicians.loc[(physicians['pop_per_pcp']>=1500) & (physicians['pop_per_pcp']<3500)]
moderately_inadequate.shape

(50, 7)

50 counties have moderately inadequate pcp coverage

In [110]:
low_inadequate = physicians.loc[physicians['pop_per_pcp']>=3500]
low_inadequate.shape

(31, 7)

31 counties have low inadequate pcp coverage

In [73]:
adequate.urban.value_counts()

Urban    9
Rural    5
Name: urban, dtype: int64

In [75]:
moderately_inadequate.urban.value_counts()

Rural    31
Urban    19
Name: urban, dtype: int64

In [76]:
low_inadequate.urban.value_counts()

Rural    21
Urban    10
Name: urban, dtype: int64

64% of counties with adequate pcp coverage are urban while 38% of counties with moderately inadequate pcp coverage are urban and 32% of counties with low inadequate pcp coverage are urban. In other words, only 9% of rural counties have adequate healthcare coverage while 24% of urban counties have adequate healthcare coverage.